In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

***
**Read the `.txt` file to dataframe**
***

In [2]:
train_data=pd.read_csv("train.txt", delimiter=",", header=None,names=['Neighbours'],index_col=False)

In [3]:
train_data['ID']=train_data['Neighbours'].apply(lambda x: x.split('\t')[0])  # Get the ID

In [4]:
train_data['Neighbours']=train_data['Neighbours'].apply(lambda x: x.split('\t')[1:])# set up the neighbours

In [5]:
train_data=train_data[["ID","Neighbours"]] # set "ID" as the index

***
**Transfer the table data to the graph data**
***

In [6]:
G = nx.DiGraph()

In [8]:
num_source=train_data.shape[0]
sink=train_data.iloc[0,]
len_sink=len(sink)

In [9]:
for i in range(num_source):
    source=int(train_data.iloc[i,0]) # the source
    sinks=train_data.iloc[i,1] # Neighbours
    len_sink=len(sinks)
    for j in range(len_sink):
        sink=sinks[j]
        G.add_node(sink)
        G.add_edge(source,sink)

nx.draw(G,with_lables=True)
plt.draw
plt.show()

In [13]:
len(G.edges)

23946602

In [14]:
len(G.nodes)

4886706